# 任务3 手写体数字识别MNIST
MNIST是机器学习届的 *Hello World*

0-9共10个数字。每个数字6000个训练图片，1000个测试图片。
每个图片为28x28灰度。

## 大纲
- 实验数据准备
- Keras实现一个CNN
- Keras Model转为Estimator

## 实验数据准备

In [ ]:
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
print("Done")

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

## 可视化数据

In [ ]:
import matplotlib.pyplot as plt
import random

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

def show_data(x, y):
    plt.figure()
    for i in range(15):
        plt.subplot(3, 5, i+1)
        index = random.randint(0, x.shape[0])
        plt.title(str(y[index]))
        plt.imshow(x[index], cmap='gray')
        plt.axis('off')

show_data(x_train, y_train)
show_data(x_test, y_test)

## 数据预处理
- 将0-255灰度值转换为0-1浮点数，并该shape为4维（低层库兼容处理彩色）
- 将0-9类别信息进行onehot编码

In [ ]:
def convert_image(x):
    x = x.reshape(x.shape[0], x.shape[1], x.shape[2], 1)
    x = x.astype('float32')
    x /= 255.0
    return x

def convert_label(y, num_classes=10):
    return tf.keras.utils.to_categorical(y, num_classes)

x_train = convert_image(x_train)
x_test = convert_image(x_test)
y_train = convert_label(y_train)
y_test = convert_label(y_test)

## Keras实现一个CNN

### 定义模型

In [ ]:
def get_model(input_shape, num_classes):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(
        input_shape=input_shape,
        name = 'my_input'))
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
                  optimizer=tf.keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

# x_train.shape的第一个维度是样本
model = get_model(x_train.shape[1:], num_classes = 10)

### 查看模型结构

In [ ]:
model.summary()

### 训练

In [ ]:
batch_size = 128
# The larger epochs, the better. Here we just use 2 for quick demo.
epochs = 1

history = model.fit(x_train, y_train, 
                    batch_size = batch_size,
                    epochs = epochs, verbose=1)
print('done')

### 推断

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# 回slides讲解
- Conv2D
- MaxPooling2D
- Dropout
- Dense
- Crossentropy

## Estimator

In [ ]:
estimator = tf.keras.estimator.model_to_estimator(
    keras_model = model)

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'my_input': x_train},
    y=y_train.astype('float32'),
    shuffle=True,
    batch_size=batch_size,
    num_epochs=epochs)

estimator = estimator.train(input_fn=train_input_fn)

### 测试

In [ ]:
# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"my_input": x_test},
    y=y_test.astype('float32'),
    num_epochs=1,
    shuffle=False)
estimator.evaluate(input_fn=test_input_fn)


# 看一下TensorBoard
Keras Model转为Estimator的好处是可以更有效的利用TensorFlow的工具和其它功能，如TensorBoard及分布式。

### 分布式环境

思考如何用RunConfig来设置Estimator到分布式TF环境。

参考https://www.tensorflow.org/api_docs/python/tf/estimator/RunConfig
